In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import sys
import warnings
import logging
import random
import time
import cv2
import math
import json
import shutil
import glob
import re
import itertools
import copy
import collections
import h5py
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as ticker
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec

from tensorflow.keras import layers, models 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, Dropout, concatenate, Input
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import jaccard_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import log_loss
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import hinge_loss
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import mean_squared_log_error





                

# --- Sanity ---
def test_util_sanity():
    print("✅ from util.ipynb")

# Define the mapping from RGB color to class index and vice-versa
COLOR_TO_CLASS = {
    (230, 25, 75): 0,       # BUILDING
    (145, 30, 180): 1,      # CLUTTER
    (60, 180, 75): 2,       # VEGETATION
    (245, 130, 48): 3,      # WATER
    (255, 255, 255): 4,     # GROUND
    (0, 130, 200): 5        # CAR
}

CLASS_TO_COLOR = {v: k for k, v in COLOR_TO_CLASS.items()}
NUM_CLASSES = len(COLOR_TO_CLASS)
COLOR_PALETTE = np.array(list(COLOR_TO_CLASS.keys()), dtype=np.uint8)

# --- Visualisation ---
def visualise_prediction(rgb, true_mask, pred_mask):
    fig, axs = plt.subplots(1, 3, figsize=(16, 5))
    axs[0].imshow(rgb)
    axs[0].set_title("RGB Image")
    axs[0].axis("off")
    axs[1].imshow(COLOR_PALETTE[true_mask])
    axs[1].set_title("True Mask")
    axs[1].axis("off")
    axs[2].imshow(COLOR_PALETTE[pred_mask])
    axs[2].set_title("Predicted Mask")
    axs[2].axis("off")
    plt.tight_layout()
    plt.show()

# --- Class Weights ---
def compute_class_weights(generator):
    pixel_counts = np.zeros(NUM_CLASSES, dtype=np.int64)

    for _, labels in generator:
        flat = np.argmax(labels, axis=-1).flatten()
        counts = np.bincount(flat, minlength=NUM_CLASSES)
        pixel_counts[:len(counts)] += counts

    total = np.sum(pixel_counts)
    weights = total / (NUM_CLASSES * np.maximum(pixel_counts, 1))
    weights = weights / np.sum(weights) * NUM_CLASSES
    return tf.constant(weights, dtype=tf.float32)

# --- Distribution Plot ---
def plot_class_distribution(generator, title="Class Distribution"):
    pixel_counts = np.zeros(NUM_CLASSES, dtype=np.int64)

    for _, labels in generator:
        if labels.size == 0:
            continue
        flat = np.argmax(labels, axis=-1).flatten()
        counts = np.bincount(flat, minlength=NUM_CLASSES)
        pixel_counts[:len(counts)] += counts

    class_names = [
        "0: Building",
        "1: Clutter",
        "2: Vegetation",
        "3: Water",
        "4: Background",
        "5: Car"
    ]

    colours = [np.array(CLASS_TO_COLOR[i]) / 255.0 for i in range(NUM_CLASSES)]

    plt.figure(figsize=(10, 5))
    bars = plt.bar(class_names, pixel_counts, color=colours, edgecolor='black')
    plt.title(title)
    plt.xlabel("Class")
    plt.ylabel("Pixel Count")
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)

    # Annotate bar values
    for bar, count in zip(bars, pixel_counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f"{count:,}", 
                 ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.show()



import tensorflow as tf
import numpy as np

# Class weights for 6 classes (0 to 5), based on imbalance
weights = np.array([0.2374, 0.2374, 0.0356, 0.2374, 0.0148, 0.2374], dtype=np.float32)

# Dice Loss
def dice_loss(y_true, y_pred, smooth=1e-6):
    # Ensure inputs are float32
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    # Flatten predictions and ground truth
    y_true_f = tf.reshape(y_true, [-1, 6])  # Shape: [batch_size * pixels, 6]
    y_pred_f = tf.reshape(y_pred, [-1, 6])
    
    # Compute intersection and union per class
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)  # Sum over pixels, per class
    denominator = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0) + smooth
    
    # Dice score per class
    dice = (2.0 * intersection + smooth) / denominator
    
    # Apply class weights and compute mean loss
    weighted_dice = weights * dice
    dice_loss = 1.0 - tf.reduce_mean(weighted_dice)
    
    return dice_loss

# Categorical Focal Loss
def categorical_focal_loss(gamma=2.0):
    def focal_loss(y_true, y_pred):
        # Ensure inputs are float32
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        
        # Clip predictions to avoid log(0)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        
        # Compute cross-entropy
        ce = -y_true * tf.math.log(y_pred)
        
        # Focal factor: (1 - p_t)^gamma
        focal_factor = tf.pow(1.0 - y_pred, gamma)
        
        # Weighted focal loss
        loss = focal_factor * ce
        
        # Mean over classes and pixels
        return tf.reduce_mean(loss)
    
    return focal_loss

# Combined Loss
def combined_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred) + categorical_focal_loss(gamma=2.0)(y_true, y_pred)

# Example: Compile model
# model = your_model  # e.g., U-Net with 6 classes
# model.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy'])